In [12]:
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt # plotting
import numpy as np # linear algebra
import os # accessing directory structure
import pandas as pd 

In [13]:
nRowsRead = 1000 
df = pd.read_csv('News_DJIA.csv', delimiter=',', nrows = nRowsRead)
df.dataframeName = 'News_DJIA.csv'
nRow, nCol = df.shape
print(f'There are {nRow} rows and {nCol} columns')

There are 1000 rows and 27 columns


In [14]:
nRowsRead = 1000 # specify 'None' if want to read whole file

df2 = pd.read_csv('DJIA_t.csv', delimiter=',', nrows = nRowsRead)
df2.dataframeName = 'DJIA_t.csv'
nRow, nCol = df2.shape
print(f'There are {nRow} rows and {nCol} columns')

There are 1000 rows and 7 columns


In [15]:
nRowsRead = 1000 # specify 'None' if want to read whole file

df3 = pd.read_csv('News_reddit.csv', delimiter=',', nrows = nRowsRead)
df3.dataframeName = 'News_reddit.csv'
nRow, nCol = df3.shape
print(f'There are {nRow} rows and {nCol} columns')

There are 1000 rows and 2 columns


In [16]:
import nltk
nltk.download('punkt')

from nltk.tokenize import word_tokenize

# create a list of columns containing text
text_columns = [col for col in df.columns if col.startswith('Top')]

for col in text_columns:
    df[col + '_tokenized'] = df[col].apply(word_tokenize)

# show the first 5 rows of the new columns
print(df[[col + '_tokenized' for col in text_columns]].head())

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\999sb\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


TypeError: expected string or bytes-like object

In [ ]:
df[text_columns] = df[text_columns].apply(lambda x: x.str.lower())

In [ ]:
from nltk.stem import WordNetLemmatizer

def lemmatize_words(text):
    lemmatizer = WordNetLemmatizer()
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()])

df[text_columns] = df[text_columns].apply(lambda x: x.apply(lemmatize_words))

In [ ]:
import nltk
from nltk.corpus import stopwords

# Download the stopwords and punkt corpora from nltk
nltk.download('stopwords')
nltk.download('punkt')

def remove_stopwords(text):
    # Create a set of English stop words
    stop_words = set(stopwords.words("english"))
    
    # Tokenize the input text into words
    words = nltk.word_tokenize(text)
    
    # Convert words to lowercase and remove stop words
    words = [word.lower() for word in words if word.lower() not in stop_words]
    
    # Join the words back into a single string
    return " ".join(words)


# Loop through all columns in the dataframe
for col in df.columns:
    # Check if the column data type is object
    if df[col].dtype == 'object':
        # Apply the remove_stopwords function to the column if the value is a string
        df[col] = df[col].apply(lambda x: remove_stopwords(x) if type(x) == str else x)

In [ ]:
import string

# Define a function to remove punctuation from a string
def remove_punctuation(text):
    # Create a string of all ASCII punctuation characters
    punctuation = string.punctuation
    # Use translate method to remove all punctuation characters from the text
    text = text.translate(str.maketrans("", "", punctuation))
    return text

# Loop through all columns in the dataframe
for col in df.columns:
    # Check if the column data type is object (e.g. string)
    if df[col].dtype == 'object':
        # Apply the remove_punctuation function to each string value in the column
        df[col] = df[col].apply(lambda x: remove_punctuation(x) if type(x) == str else x)

In [ ]:
import re

def remove_non_textual(text):
    text = re.sub(r"http\S+", "", text)  # remove URLs
    text = re.sub(r"\d+", "", text)  # remove numbers
    text = re.sub(r"[^\w\s]", "", text)  # remove special characters
    return text

for col in df.columns:
    if df[col].dtype == 'object':
        df[col] = df[col].apply(lambda x: remove_non_textual(x) if type(x) == str else x)

In [ ]:
from nltk import FreqDist

def remove_corpus_specific_words(text, threshold):
    words = nltk.word_tokenize(text)
    fdist = FreqDist(words)
    stop_words = [word for word, count in fdist.items() if count > threshold]
    words = [word for word in words if word not in stop_words]
    return " ".join(words)

for col in df.columns:
    if df[col].dtype == 'object':
        df[col] = df[col].apply(lambda x: remove_corpus_specific_words(x, 100) if type(x) == str else x)

In [ ]:
columns = ['Label','Top1_tokenized', 'Top2_tokenized', 'Top3_tokenized',           'Top4_tokenized', 'Top5_tokenized', 'Top6_tokenized', 'Top7_tokenized',           'Top8_tokenized', 'Top9_tokenized', 'Top10_tokenized',           'Top11_tokenized', 'Top12_tokenized', 'Top13_tokenized',           'Top14_tokenized', 'Top15_tokenized', 'Top16_tokenized',           'Top17_tokenized', 'Top18_tokenized', 'Top19_tokenized',           'Top20_tokenized', 'Top21_tokenized', 'Top22_tokenized',           'Top23_tokenized', 'Top24_tokenized', 'Top25_tokenized']
df = df[columns]

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

def tfidf_features(dataframe):
    tfidf = TfidfVectorizer(min_df=2)  # minimum document frequency to consider a word
    features = tfidf.fit_transform(dataframe)
    feature_names = tfidf.get_feature_names()
    return features, feature_names

# Apply TF-IDF to all columns with text data
for col in df.columns:
    if df[col].dtype == 'object':
        text = df[col].apply(str)  # convert pandas series to list of strings
        features, feature_names = tfidf_features(text)
        df_tfidf = pd.DataFrame(features.todense(), columns=feature_names)
        df = pd.concat([df, df_tfidf], axis=1)

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer



# extract the text data
text_data = df[['Top1_tokenized', 'Top2_tokenized', 'Top3_tokenized',           'Top4_tokenized', 'Top5_tokenized', 'Top6_tokenized', 'Top7_tokenized',           'Top8_tokenized', 'Top9_tokenized', 'Top10_tokenized',           'Top11_tokenized', 'Top12_tokenized', 'Top13_tokenized',           'Top14_tokenized', 'Top15_tokenized', 'Top16_tokenized',           'Top17_tokenized', 'Top18_tokenized', 'Top19_tokenized',           'Top20_tokenized', 'Top21_tokenized', 'Top22_tokenized',           'Top23_tokenized', 'Top24_tokenized', 'Top25_tokenized']]

# combine all the text data into a single string for each row
text_data = text_data.apply(lambda x: " ".join(str(x) for x in x), axis=1)

# perform count vectorization to convert text into numerical data
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(text_data)

# extract the target variable
y = df["Label"]

In [ ]:
from sklearn.model_selection import train_test_split

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Split the training data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier


# Define the models to be trained
models = [LogisticRegression(), SVC(), RandomForestClassifier(), GradientBoostingClassifier(), KNeighborsClassifier(), DecisionTreeClassifier()]
model_names = ["Logistic Regression", "Support Vector Classifier", "Random Forest", "Gradient Boosting", "K-Nearest Neighbors", "Decision Tree"]

results=[]
# Train each model and get predictions
for model, model_name in zip(models, model_names):
    model.fit(train_data, train_labels)
    predictions = model.predict(test_data)
    
    # Calculate evaluation metrics for each model
    accuracy = accuracy_score(test_labels, predictions)
    precision = precision_score(test_labels, predictions)
    recall = recall_score(test_labels, predictions)
    f1 = f1_score(test_labels, predictions)
    
    # Store the results of each model in the results list
    results.append((model_name, accuracy, precision, recall, f1))
    
# Print the results
for result in results:
    print("Model: {}\nAccuracy: {:.2f}%\nPrecision: {:.2f}%\nRecall: {:.2f}%\nF1 Score: {:.2f}%\n".format(result[0], result[1]*100, result[2]*100, result[3]*100, result[4]*100))